#Stage Two - Data Cleaning and Feature Engineering
#End Project: What Makes a Painting Display-Worthy at the Met Museum?
#Predicting Gallery Display Likelihood

This notebook takes the raw dataset from notebook 01 and prepares it for analysis and modelling.

#The steps to be taken are:
1.Load raw data and remove duplicates
2.Handle missing values
3.Clean and standardise existing columns
4.Engineer new features
5.Encode categorical variables
6.Save clean dataset for further analysis


#One - Setup and Load The Full Data

In [2]:
import pandas as pd
import numpy as np
import re
import os

os.makedirs("data/clean", exist_ok=True)

df = pd.read_csv("data/raw/met_paintings_raw.csv")
print(f"Loaded: {len(df):,} rows x {df.shape[1]} columns")

Loaded: 9,269 rows x 23 columns


#Two - Remove Any Duplicates

The merge step in notebook 01 could have introduced duplicate rows so want to remove now

In [3]:
dupes = df.duplicated(subset="object_id", keep="first").sum()
print(f"Duplicate object_id rows: {dupes}")

df = df.drop_duplicates(subset="object_id", keep="first").reset_index(drop=True)
print(f"After dedup: {len(df):,} rows")

Duplicate object_id rows: 264
After dedup: 9,005 rows


#Three -  Review The Initial Data Profile

Quick overview data before making changes. Best practice.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9005 entries, 0 to 9004
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   object_id           9005 non-null   int64  
 1   object_id.1         9005 non-null   int64  
 2   title               6775 non-null   object 
 3   artist_name         8021 non-null   object 
 4   artist_nationality  8021 non-null   object 
 5   artist_begin_date   8021 non-null   object 
 6   artist_end_date     8021 non-null   object 
 7   object_begin_date   9005 non-null   int64  
 8   object_end_date     9005 non-null   int64  
 9   medium              9001 non-null   object 
 10  dimensions          8991 non-null   object 
 11  department          9005 non-null   object 
 12  culture             4424 non-null   object 
 13  period              2731 non-null   object 
 14  credit_line         9000 non-null   object 
 15  accession_year      8967 non-null   float64
 16  is_pub

In [5]:
# Missing values summary
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)
missing_df = pd.DataFrame({"missing": missing, "pct": missing_pct})
print(missing_df[missing_df["missing"] > 0].sort_values("pct", ascending=False))

                    missing   pct
gallery_number         7626  84.7
period                 6274  69.7
culture                4581  50.9
title                  2230  24.8
width_cm               1879  20.9
height_cm              1864  20.7
tags                   1764  19.6
artist_name             984  10.9
artist_nationality      984  10.9
artist_begin_date       984  10.9
artist_end_date         984  10.9
accession_year           38   0.4
dimensions               14   0.2
credit_line               5   0.1
medium                    4   0.0


In [6]:
# Target variable check
print(f"Target distribution:")
print(df["is_on_display"].value_counts())
print(f"Display rate: {df['is_on_display'].mean()*100:.1f}%")

Target distribution:
is_on_display
0    7626
1    1379
Name: count, dtype: int64
Display rate: 15.3%


#Four - Cleaning The Existing Columns

#Four - Part One: Parse Dimensions into Numeric Height and Width

Where the API gave structured measurements, I now have `height_cm` and `width_cm`.  
For rows where those are missing can try to parse the `dimensions` text string as a backup.

In [7]:
# Check some dimension strings to understand the data format
print("Sample dimension strings are:")
for val in df["dimensions"].dropna().sample(10, random_state=42).values:
    print(f"  {val}")

Sample dimension strings are:
  Image: 31 1/2 × 12 3/8 in. (80 × 31.5 cm)
Overall with mounting: 65 3/8 × 13 1/4 in. (166 × 33.7 cm)
Overall with knobs: 65 3/8 × 15 1/16 in. (166 × 38.2 cm)
  Image: 27 × 27 in. (68.6 × 68.6 cm)
Framed: 33 3/4 in. × 31 3/4 in. × 3/4 in. (85.7 × 80.6 × 1.9 cm)
  93 3/4 x 57 1/4 in. (238.1 x 145.4 cm)
  18 3/8 × 21 3/4 in. (46.7 × 55.2 cm)
  Oval, 25 1/4 x 20 3/4 in. (64.1 x 52.7 cm)
  Image (each leaf): 5 5/16 × 5 1/8 in. (13.5 × 13 cm)
Album: 7 1/8 × 6 1/8 × 1 3/16 in. (18.1 × 15.6 × 3 cm)
  38 1/16 x 9 15/16 in. (96.7 x 25.2 cm)
  90 7/8 x 64 3/4 in. (230.8 x 164.5 cm)
  66 × 86 1/8 in. (167.6 × 218.8 cm)
  13 1/8 x 9 1/8 in. (33.3 x 23.2 cm)


In [8]:
def parse_dimensions_cm(dim_str):
    """Try to extract height and width in cm from a dimension string.
    
    Common formats:
      '25 x 30 cm'
      '25 × 30 cm'
      '10 x 15 in. (25.4 x 38.1 cm)'
      'Overall: 25 x 30 cm'
    """
    if pd.isna(dim_str):
        return None, None
    
    # Trying to find cm measurements in parentheses first - most reliable route
    cm_match = re.search(r'\(([\d.]+)\s*[x×]\s*([\d.]+)\s*cm\)', dim_str)
    if cm_match:
        return float(cm_match.group(1)), float(cm_match.group(2))
    
    # Try plain cm format without parentheses
    cm_match = re.search(r'([\d.]+)\s*[x×]\s*([\d.]+)\s*cm', dim_str)
    if cm_match:
        return float(cm_match.group(1)), float(cm_match.group(2))

    # If no cm found, try to parse inches and convert to cm
    in_match = re.search(r'([\d.]+)\s*[x×]\s*([\d.]+)\s*in', dim_str)
    if in_match:
        return float(in_match.group(1)) * 2.54, float(in_match.group(2)) * 2.54

    return None, None

# Testing the parser
test_cases = [
    "10 x 15 in. (25.4 x 38.1 cm)",
    "25 × 30 cm",
    "Overall: 25 x 30 cm",
]
for t in test_cases:
    h, w = parse_dimensions_cm(t)
    print(f"  '{t}' -> height={h}, width={w}")

  '10 x 15 in. (25.4 x 38.1 cm)' -> height=25.4, width=38.1
  '25 × 30 cm' -> height=25.0, width=30.0
  'Overall: 25 x 30 cm' -> height=25.0, width=30.0


In [9]:
# Filling missing height/width from dimension string where it is possible
before_h = df["height_cm"].notna().sum()
before_w = df["width_cm"].notna().sum()

mask = df["height_cm"].isna()
parsed = df.loc[mask, "dimensions"].apply(lambda x: pd.Series(parse_dimensions_cm(x)))
df.loc[mask, "height_cm"] = parsed[0].values
df.loc[mask, "width_cm"] = parsed[1].values

after_h = df["height_cm"].notna().sum()
after_w = df["width_cm"].notna().sum()

print(f"height_cm: {before_h:,} -> {after_h:,} (recovered {after_h - before_h:,})")
print(f"width_cm:  {before_w:,} -> {after_w:,} (recovered {after_w - before_w:,})")

height_cm: 7,141 -> 8,951 (recovered 1,810)
width_cm:  7,126 -> 8,920 (recovered 1,794)


#Four - Part Two: Cleaning Medium

The `medium` column has some detailed descriptions like *"Oil on canvas"* or *"Ink and color on silk"*.  
So need to extract a simplified primary medium for modelling.

In [10]:
# Checking most common medium values
print("Top 20 mediums are:")
print(df["medium"].value_counts().head(20))

Top 20 mediums are:
medium
Oil on canvas                                                        2482
Oil on wood                                                           476
Hanging scroll; ink and color on paper                                406
Hanging scroll; ink and color on silk                                 401
Hanging scroll; ink on paper                                          267
Ink, opaque watercolor, and gold on paper                             222
Tempera on wood, gold ground                                          154
Album leaf; ink and color on silk                                     135
Ink and opaque watercolor on paper                                    123
Acrylic on canvas                                                      98
Handscroll; ink and color on silk                                      96
Oil on paper, laid down on canvas                                      89
Folding fan mounted as an album leaf; ink and color on alum paper      82
Tempera and

In [11]:
def extract_primary_medium(medium_str):
    """Extract a simplified primary medium category."""
    if pd.isna(medium_str):
        return "Unknown"
    
    m = medium_str.lower()
    
    if "oil" in m:
        return "Oil"
    elif "watercolor" in m or "watercolour" in m:
        return "Watercolour"
    elif "acrylic" in m:
        return "Acrylic"
    elif "tempera" in m:
        return "Tempera"
    elif "ink" in m:
        return "Ink"
    elif "gouache" in m:
        return "Gouache"
    elif "pastel" in m:
        return "Pastel"
    elif "fresco" in m:
        return "Fresco"
    elif "enamel" in m:
        return "Enamel"
    else:
        return "Other"

df["primary_medium"] = df["medium"].apply(extract_primary_medium)
print("Primary medium distribution:")
print(df["primary_medium"].value_counts())

Primary medium distribution:
primary_medium
Oil            3874
Ink            3240
Watercolour     639
Other           582
Tempera         406
Acrylic         214
Fresco           23
Enamel           13
Gouache           9
Unknown           4
Pastel            1
Name: count, dtype: int64


#Four - Part Three: Clean Support (Canvas, Paper, etc.)

Because the surface the painting is on (canvas, silk, paper, wood, etc.) could also influence display decisions.

In [12]:
def extract_support(medium_str):
    """Extract the painting support/surface from the medium string."""
    if pd.isna(medium_str):
        return "Unknown"
    
    m = medium_str.lower()
    
    if "canvas" in m:
        return "Canvas"
    elif "silk" in m:
        return "Silk"
    elif "paper" in m:
        return "Paper"
    elif "wood" in m or "panel" in m:
        return "Wood/Panel"
    elif "copper" in m:
        return "Copper"
    elif "ivory" in m:
        return "Ivory"
    elif "linen" in m:
        return "Linen"
    elif "gold" in m:
        return "Gold"
    else:
        return "Other"

df["support"] = df["medium"].apply(extract_support)
print("Support distribution:")
print(df["support"].value_counts())

Support distribution:
support
Canvas        3137
Paper         2958
Silk          1213
Wood/Panel    1036
Other          552
Gold            49
Copper          34
Linen           22
Unknown          4
Name: count, dtype: int64


#Four - Part Four: Cleaning the Artist's Nationality

Nationality has many unique values which could cause issues. So should group smaller categories into regions to avoid
very sparse categories.

In [13]:
# Check the current nationality distribution first
print(f"Unique nationalities: {df['artist_nationality'].nunique()}")
print("\nTop 20:")
print(df["artist_nationality"].value_counts().head(20))

Unique nationalities: 200

Top 20:
artist_nationality
Chinese                   1336
American                  1150
French                    1033
Japanese                  1009
Italian                    639
                           576
Dutch                      254
British                    220
German                     165
Netherlandish              164
Spanish                    143
 |Chinese                  141
Indian                     132
Flemish                     98
Chinese|                    78
Japanese|Japanese           52
American, born Germany      41
American, born Russia       39
Russian                     33
American                    27
Name: count, dtype: int64


In [14]:
def group_nationality(nat_str):
    """Group nationalities into broader regions.
    
    Keep the most common nations as-is, group the rest into regions.
    Adjust thresholds after reviewing the distribution above.
    """
    if pd.isna(nat_str):
        return "Unknown"
    
    n = nat_str.strip().lower()
    
    # Handle pipe-separated values (multiple artists)
    if "|" in n:
        n = n.split("|")[0].strip()
    
    # Keep the most common nationalities as-is
    keep_as_is = [
        "american", "chinese", "french", "italian", "japanese",
        "british", "dutch", "german", "spanish", "flemish",
        "indian", "korean"
    ]
    for nat in keep_as_is:
        if nat in n:
            return nat.title()
    
    # Group the rest by region
    european = ["austrian", "belgian", "swiss", "russian", "swedish",
                "norwegian", "danish", "portuguese", "greek", "polish",
                "hungarian", "czech", "irish", "scottish", "finnish"]
    if any(e in n for e in european):
        return "Other European"
    
    latin_american = ["mexican", "brazilian", "cuban", "colombian",
                      "argentine", "peruvian", "venezuelan", "chilean"]
    if any(la in n for la in latin_american):
        return "Latin American"
    
    asian = ["thai", "tibetan", "nepalese", "vietnamese", "indonesian",
             "burmese", "cambodian"]
    if any(a in n for a in asian):
        return "Other Asian"
    
    return "Other"

df["nationality_group"] = df["artist_nationality"].apply(group_nationality)
print("Grouped nationality distribution are:")
print(df["nationality_group"].value_counts())

Grouped nationality distribution are:
nationality_group
Chinese           1450
American          1415
Japanese          1078
French            1065
Other             1023
Unknown            984
Italian            657
Dutch              255
British            255
Other European     192
German             179
Spanish            145
Indian             137
Flemish            104
Latin American      43
Korean              19
Other Asian          4
Name: count, dtype: int64


#Four - Part Five: Cleaning Accession Year. I.e. year admitted to Met Museum

Convert accession year to numeric and handle any invalid values.

In [15]:
df["accession_year"] = pd.to_numeric(df["accession_year"], errors="coerce")
print(f"Accession year range: {df['accession_year'].min():.0f} to {df['accession_year'].max():.0f}")
print(f"Missing: {df['accession_year'].isna().sum()}")

Accession year range: 1871 to 2023
Missing: 38


#Four - Part Six: Grouping the Culture Column

The culture column (e.g. 'French, 17th century') contains real signal about display likelihood and is distinct from both artist nationality and department. It has high cardinality so needs grouping before it can be used as a model feature.

In [16]:
def group_culture(culture_str):
    """Extract a broad cultural region from the culture field."""
    if pd.isna(culture_str) or culture_str.strip() == "":
        return "Unknown"
    c = culture_str.lower()
    if any(x in c for x in ["french", "france"]):
        return "French"
    elif any(x in c for x in ["italian", "italy", "roman", "venetian", "florentine"]):
        return "Italian"
    elif any(x in c for x in ["dutch", "netherlands", "holland"]):
        return "Dutch"
    elif any(x in c for x in ["flemish", "belgian"]):
        return "Flemish"
    elif any(x in c for x in ["british", "english", "welsh", "scottish"]):
        return "British"
    elif any(x in c for x in ["american", "united states"]):
        return "American"
    elif any(x in c for x in ["chinese", "china"]):
        return "Chinese"
    elif any(x in c for x in ["japanese", "japan"]):
        return "Japanese"
    elif any(x in c for x in ["spanish", "spain"]):
        return "Spanish"
    elif any(x in c for x in ["german", "germany", "austrian"]):
        return "German/Austrian"
    elif any(x in c for x in ["indian", "india"]):
        return "Indian"
    else:
        return "Other"

df["culture_group"] = df["culture"].apply(group_culture)
print("Culture group distribution:")
print(df["culture_group"].value_counts())

Culture group distribution:
culture_group
Unknown            4581
Chinese            1983
Japanese           1431
Indian              682
Other               276
Italian              30
French               11
American              5
Spanish               3
Flemish               1
German/Austrian       1
British               1
Name: count, dtype: int64


In [17]:
# Checking display rate by culture group to confirm predictive value
print("Display rate by culture group:")
print(df.groupby("culture_group")["is_on_display"].mean().round(3).sort_values(ascending=False))

Display rate by culture group:
culture_group
Flemish            1.000
German/Austrian    1.000
Italian            0.300
Unknown            0.283
French             0.182
Other              0.069
Japanese           0.016
Chinese            0.014
Indian             0.001
American           0.000
British            0.000
Spanish            0.000
Name: is_on_display, dtype: float64


#Five - Feature Engineering

Creating new features that could be predictive of display status.

#Five - Part One: The Paintings Age

In [18]:
#Using the midpoint of begin and end date as estimated creation date
df["creation_year"] = ((df["object_begin_date"] + df["object_end_date"]) / 2).round(0)
df["painting_age"] = 2026 - df["creation_year"]

print(f"Painting age range is: {df['painting_age'].min():.0f} to {df['painting_age'].max():.0f} years")
print(f"The median age of paintings is: {df['painting_age'].median():.0f} years")

Painting age range is: 4 to 2026 years
The median age of paintings is: 200 years


In [19]:
#Fixing an issue: creation_year of 0 means unknown, not year zero
#Using department-level median rather than global median for accuracy
zero_date_mask = df["creation_year"] == 0
print(f"Paintings with unknown date (creation_year = 0): {zero_date_mask.sum()}")

dept_year_median = df.groupby("department")["creation_year"].transform(
    lambda x: x[x != 0].median()
)
global_year_median = df.loc[~zero_date_mask, "creation_year"].median()

df.loc[zero_date_mask, "creation_year"] = dept_year_median[zero_date_mask].fillna(global_year_median)
df.loc[zero_date_mask, "painting_age"] = 2026 - df.loc[zero_date_mask, "creation_year"]

print(f"Filled using department medians (global fallback: {global_year_median:.0f})")
print(df.groupby("department")["creation_year"].median().sort_values())

Paintings with unknown date (creation_year = 0): 9
Filled using department medians (global fallback: 1827)
department
Arms and Armor                               1583.0
European Paintings                           1748.0
Robert Lehman Collection                     1750.0
Asian Art                                    1754.0
Arts of Africa, Oceania, and the Americas    1800.0
Musical Instruments                          1813.0
European Sculpture and Decorative Arts       1903.0
Drawings and Prints                          1912.0
Modern and Contemporary Art                  1956.0
Photographs                                  1960.0
Islamic Art                                  1979.0
Name: creation_year, dtype: float64


#Five - Part One (b): Assigning Era Buckets From Creation Year

In [20]:
def assign_era(year):
    """Group creation year into broad historical periods.
    
    Each painting is placed into a named era based on when it was made.
    For example, a painting with creation_year of 1650 falls between 1600
    and 1800, so it is labelled Baroque and Enlightenment.
    This gives the model a categorical handle on time rather than a raw number.
    """
    if pd.isna(year) or year < 0:
        return "Ancient or Unknown"
    elif year < 500:
        return "Ancient"
    elif year < 1400:
        return "Medieval"
    elif year < 1600:
        return "Renaissance"
    elif year < 1800:
        return "Baroque and Enlightenment"
    elif year < 1900:
        return "19th Century"
    elif year < 1950:
        return "Early Modern"
    else:
        return "Contemporary"

df["era"] = df["creation_year"].apply(assign_era)
print("Era distribution:")
print(df["era"].value_counts())

Era distribution:
era
Baroque and Enlightenment    2172
19th Century                 1828
Contemporary                 1644
Early Modern                 1502
Renaissance                  1284
Medieval                      571
Ancient                         4
Name: count, dtype: int64


In [21]:
#Checking display rate by era to confirm predictive value
print("Display rate by era:")
print(df.groupby("era")["is_on_display"].mean().round(3).sort_values(ascending=False))

Display rate by era:
era
Renaissance                  0.276
19th Century                 0.220
Baroque and Enlightenment    0.203
Medieval                     0.123
Early Modern                 0.050
Contemporary                 0.023
Ancient                      0.000
Name: is_on_display, dtype: float64


#Five - Part Two: How Many Years in Met's Collection

In [22]:
df["years_in_collection"] = 2026 - df["accession_year"]
print(f"Years in collection range: {df['years_in_collection'].min():.0f} to {df['years_in_collection'].max():.0f}")

Years in collection range: 3 to 155


#Five - Part Two (b): Flagging Recently Acquired Paintings

In [23]:
# Paintings acquired from 1990 onward are considered recently acquired
# These may be displayed more actively to reflect current collection priorities
df["recently_acquired"] = (df["accession_year"] >= 1990).astype(int)

print(f"Recently acquired (1990 onwards): {df['recently_acquired'].sum():,} ({df['recently_acquired'].mean()*100:.1f}%)")

Recently acquired (1990 onwards): 2,959 (32.9%)


In [24]:
# Checking display rate by acquisition recency
print("Display rate by acquisition period:")
print(df.groupby("recently_acquired")["is_on_display"].mean().round(3))

Display rate by acquisition period:
recently_acquired
0    0.168
1    0.122
Name: is_on_display, dtype: float64


#Five - Part Three: Assessing The Paintins Size In terms of Area

In [25]:
df["area_cm2"] = df["height_cm"] * df["width_cm"]
print(f"Area coverage: {df['area_cm2'].notna().sum():,} / {len(df):,} ({df['area_cm2'].notna().mean()*100:.1f}%)")
print(f"Median area: {df['area_cm2'].median():,.0f} cm2")

Area coverage: 8,920 / 9,005 (99.1%)
Median area: 3,536 cm2


#Five - Part Three (a): Log Transforming Skewed Size Features

Physical measurements like area_cm2 are heavily right-skewed due to a small number of very large works such as murals. A log transform brings the distribution closer to normal and makes the model less sensitive to extreme values. log1p is used (meaning log(x+1)) to safely handle any zero values.

In [26]:
# Log transforming skewed size features to reduce the effect of extreme outliers
df["log_area_cm2"] = np.log1p(df["area_cm2"])
df["log_height_cm"] = np.log1p(df["height_cm"])
df["log_width_cm"] = np.log1p(df["width_cm"])

print("Skewness before and after log transform:")
for col, log_col in [("area_cm2", "log_area_cm2"), ("height_cm", "log_height_cm"), ("width_cm", "log_width_cm")]:
    print(f"  {col}: {df[col].skew():.2f} -> {df[log_col].skew():.2f}")

Skewness before and after log transform:
  area_cm2: 21.81 -> -0.04
  height_cm: 1.53 -> -0.32
  width_cm: 6.90 -> 0.62


#Five - Part Three (b): Calculating Aspect Ratio and Orientation

In [27]:
# Aspect ratio captures shape which affects where a painting can hang
df["aspect_ratio"] = (df["width_cm"] / df["height_cm"]).round(3)

# Portrait orientation means height is greater than width
df["is_portrait_orientation"] = (df["height_cm"] > df["width_cm"]).astype(int)

print(f"Aspect ratio range: {df['aspect_ratio'].min():.2f} to {df['aspect_ratio'].max():.2f}")
print(f"Median aspect ratio: {df['aspect_ratio'].median():.2f}")
print(f"Portrait orientation: {df['is_portrait_orientation'].sum():,} ({df['is_portrait_orientation'].mean()*100:.1f}%)")
print(f"Landscape orientation: {(df['is_portrait_orientation'] == 0).sum():,} ({(1 - df['is_portrait_orientation'].mean())*100:.1f}%)")

Aspect ratio range: 0.01 to 80.92
Median aspect ratio: 0.90
Portrait orientation: 4,791 (53.2%)
Landscape orientation: 4,214 (46.8%)


In [28]:
#Checking display rate by orientation
print("Display rate by orientation:")
print(df.groupby("is_portrait_orientation")["is_on_display"].mean().round(3))

Display rate by orientation:
is_portrait_orientation
0    0.124
1    0.178
Name: is_on_display, dtype: float64


#Five - Part Three (c): Analytical Summaries by Group

In [29]:
#These summaries inform the EDA and Tableau dashboards
print("Median area (cm2) by department:")
print(df.groupby("department")["area_cm2"].median().sort_values(ascending=False).round(0))

Median area (cm2) by department:
department
Arts of Africa, Oceania, and the Americas    33280.0
Modern and Contemporary Art                   8354.0
Islamic Art                                   7434.0
Arms and Armor                                6486.0
European Sculpture and Decorative Arts        6080.0
European Paintings                            3857.0
Robert Lehman Collection                      2515.0
Asian Art                                     1981.0
Photographs                                   1904.0
Drawings and Prints                            764.0
Musical Instruments                             99.0
Name: area_cm2, dtype: float64


In [30]:
print("Median area (cm2) by primary medium:")
print(df.groupby("primary_medium")["area_cm2"].median().sort_values(ascending=False).round(0))

print("\nMedian creation year by department:")
print(df.groupby("department")["creation_year"].median().sort_values().round(0))

Median area (cm2) by primary medium:
primary_medium
Acrylic        27027.0
Enamel         13950.0
Fresco          8894.0
Oil             5293.0
Ink             3022.0
Other           2260.0
Tempera         1981.0
Pastel          1547.0
Gouache         1120.0
Watercolour      330.0
Unknown            NaN
Name: area_cm2, dtype: float64

Median creation year by department:
department
Arms and Armor                               1583.0
European Paintings                           1748.0
Robert Lehman Collection                     1750.0
Asian Art                                    1754.0
Arts of Africa, Oceania, and the Americas    1800.0
Musical Instruments                          1813.0
European Sculpture and Decorative Arts       1903.0
Drawings and Prints                          1912.0
Modern and Contemporary Art                  1956.0
Photographs                                  1960.0
Islamic Art                                  1979.0
Name: creation_year, dtype: float64


In [31]:
#Displaying rate by size band using quartile cuts
df["size_band"] = pd.qcut(df["area_cm2"], q=4, labels=["Small", "Medium", "Large", "Very Large"])

print("Display rate by size band:")
print(df.groupby("size_band")["is_on_display"].mean().round(3))

print("\nPainting count by size band:")
print(df["size_band"].value_counts().sort_index())

Display rate by size band:
size_band
Small         0.093
Medium        0.167
Large         0.167
Very Large    0.189
Name: is_on_display, dtype: float64

Painting count by size band:
size_band
Small         2230
Medium        2230
Large         2232
Very Large    2228
Name: count, dtype: int64


/var/folders/8w/jbgrr0fd4jvbmhsmhc4cb5400000gn/T/ipykernel_21306/2466180041.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby("size_band")["is_on_display"].mean().round(3))


#Five - Part Four: Checking Scracity Through Artist Work Count

How many paintings does this artist have in The Met's collection?  How rare is the work? 
Artists with more works may be more prominent (or have larger output), which could affect display decisions.

In [32]:
artist_counts = df["artist_name"].value_counts()
df["artist_work_count"] = df["artist_name"].map(artist_counts).fillna(0).astype(int)

print(f"Artist work count range is: {df['artist_work_count'].min()} to {df['artist_work_count'].max()}")
print(f"\nTop ten most represented artists in the collection are:")
print(artist_counts.head(10))

Artist work count range is: 0 to 388

Top ten most represented artists in the collection are:
artist_name
Unidentified artist     388
Xie Zhiliu              344
Shibata Zeshin           80
Bhadrabahu               71
Katsushika Hokusai       60
Kawanabe Kyōsai 河鍋暁斎     49
Qi Baishi                47
Unidentified             44
Claude Monet             40
Pablo Picasso            37
Name: count, dtype: int64


In [33]:
#Fixing an issue from above in that Unidentified artists are not one person, so their work count is meaningless and confusing. So:
unknown_labels = ["Unidentified artist", "Unidentified", ""]
df.loc[df["artist_name"].isin(unknown_labels) | df["artist_name"].isna(), "artist_work_count"] = 0

#Creatting a binary feature - as whether the artist is known could itself predict display and unknown less so
df["has_known_artist"] = (~df["artist_name"].isin(unknown_labels) & df["artist_name"].notna()).astype(int)

print(f"Paintings with known artist total: {df['has_known_artist'].sum():,} ({df['has_known_artist'].mean()*100:.1f}%)")
print(f"Artist work count range (after the new fix): {df['artist_work_count'].min()} to {df['artist_work_count'].max()}")

Paintings with known artist total: 7,589 (84.3%)
Artist work count range (after the new fix): 0 to 344


#Five - Part Four (b): Flagging Collaborative Works

Some paintings are attributed to multiple artists using a pipe separator in artist_name (e.g. 'Artist A|Artist B'). The artist_work_count figure is unreliable for these rows since it counts the full pipe-separated string as a single entity. Adding a binary flag to capture this.

In [34]:
df["is_collaborative_work"] = df["artist_name"].fillna("").str.contains("|", regex=False).astype(int)

print(f"Collaborative works: {df['is_collaborative_work'].sum():,} ({df['is_collaborative_work'].mean()*100:.1f}%)")
print(f"Display rate for collaborative works: {df[df['is_collaborative_work']==1]['is_on_display'].mean():.3f}")
print(f"Display rate for single-artist works: {df[df['is_collaborative_work']==0]['is_on_display'].mean():.3f}")

Collaborative works: 389 (4.3%)
Display rate for collaborative works: 0.046
Display rate for single-artist works: 0.158


#Five - Part Five: Introducing Tag-Based Features

Extract binary features for the most common subject tags (e.g. Portraits, Landscapes, etc.).

In [35]:
#Finding the most common tags across all paintings in the collection
all_tags = df["tags"].dropna().str.split("|").explode()
all_tags = all_tags[all_tags.str.strip() != ""]
print("Top tweenty tags are:")
print(all_tags.value_counts().head(20))

Top tweenty tags are:
tags
Men                  2025
Women                1480
Landscapes           1005
Portraits             893
Mountains             403
Trees                 341
Flowers               336
Birds                 333
Horses                325
Abstraction           308
Buddhism              283
Boats                 264
Dogs                  199
Madonna and Child     180
Christ                172
Still Life            170
Virgin Mary           166
Angels                157
Female Nudes          155
Rivers                155
Name: count, dtype: int64


In [36]:
#Creating binary columns for the top tags
top_tags = all_tags.value_counts().head(10).index.tolist()
print(f"Creating binary features for: {top_tags}")

for tag in top_tags:
    col_name = f"tag_{tag.lower().replace(' ', '_')}"
    df[col_name] = df["tags"].fillna("").str.contains(tag, case=False, regex=False).astype(int)

# Also creating a total tag count
df["tag_count"] = df["tags"].fillna("").apply(lambda x: len([t for t in x.split("|") if t.strip()]))

print(f"\nTag count distribution is:")
print(df["tag_count"].describe())

Creating binary features for: ['Men', 'Women', 'Landscapes', 'Portraits', 'Mountains', 'Trees', 'Flowers', 'Birds', 'Horses', 'Abstraction']

Tag count distribution is:
count    9005.000000
mean        1.971349
std         1.590754
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        16.000000
Name: tag_count, dtype: float64


#Five - Part Six: Credit Line Feature Check

How was painting was acquired - i.e. gift, purchase, bequest) may relate to display decisions.

In [37]:
def extract_acquisition_type(credit_str):
    """Extract how the painting was acquired from the credit line."""
    if pd.isna(credit_str):
        return "Unknown"
    
    c = credit_str.lower()
    
    if "bequest" in c:
        return "Bequest"
    elif "gift" in c:
        return "Gift"
    elif "purchase" in c or "fund" in c:
        return "Purchase/Fund"
    elif "rogers" in c or "fletcher" in c:
        return "Purchase/Fund"
    elif "lent" in c or "loan" in c:
        return "Loan"
    else:
        return "Other"

df["acquisition_type"] = df["credit_line"].apply(extract_acquisition_type)
print("Acquisition type distribution:")
print(df["acquisition_type"].value_counts())

Acquisition type distribution:
acquisition_type
Gift             4940
Purchase/Fund    1794
Bequest          1709
Other             557
Unknown             5
Name: count, dtype: int64


#Five - Part Six (b): Collapsing Rare Categories

Categorical columns with very small groups (under 50 paintings) cause problems for the model as it sees too few examples to learn reliably. Checking all categorical columns and collapsing anything below the threshold into 'Other'.

In [38]:
#Collapsing rare categories to avoid noisy low-count groups in the model
rare_threshold = 50

for col in ["primary_medium", "support", "nationality_group", "acquisition_type", "culture_group"]:
    if col in df.columns:
        counts = df[col].value_counts()
        rare = counts[counts < rare_threshold]
        if len(rare) > 0:
            print(f"\n{col} -- rare categories collapsed into Other:")
            print(rare)
            df[col] = df[col].apply(lambda x: "Other" if x in rare.index else x)
        else:
            print(f"{col}: no rare categories found")


primary_medium -- rare categories collapsed into Other:
primary_medium
Fresco     23
Enamel     13
Gouache     9
Unknown     4
Pastel      1
Name: count, dtype: int64

support -- rare categories collapsed into Other:
support
Gold       49
Copper     34
Linen      22
Unknown     4
Name: count, dtype: int64

nationality_group -- rare categories collapsed into Other:
nationality_group
Latin American    43
Korean            19
Other Asian        4
Name: count, dtype: int64

acquisition_type -- rare categories collapsed into Other:
acquisition_type
Unknown    5
Name: count, dtype: int64

culture_group -- rare categories collapsed into Other:
culture_group
Italian            30
French             11
American            5
Spanish             3
Flemish             1
German/Austrian     1
British             1
Name: count, dtype: int64


#Six - Handling The Remaining Missing Values

In [39]:
#Checking what the missing values are after feature engineering
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)
missing_df = pd.DataFrame({"missing": missing, "pct": missing_pct})
print("Remaining missing values:")
print(missing_df[missing_df["missing"] > 0].sort_values("pct", ascending=False))

Remaining missing values:
                     missing   pct
gallery_number          7626  84.7
period                  6274  69.7
culture                 4581  50.9
title                   2230  24.8
tags                    1764  19.6
artist_nationality       984  10.9
artist_begin_date        984  10.9
artist_end_date          984  10.9
artist_name              984  10.9
aspect_ratio              85   0.9
log_width_cm              85   0.9
log_area_cm2              85   0.9
area_cm2                  85   0.9
size_band                 85   0.9
width_cm                  85   0.9
log_height_cm             54   0.6
height_cm                 54   0.6
years_in_collection       38   0.4
accession_year            38   0.4
dimensions                14   0.2
credit_line                5   0.1
medium                     4   0.0


In [40]:
#Flagging unknown accession year before filling it
#Paintings with no accession year on record are almost always very early acquisitions
#where records were not kept the absence of data is itself informative
df["accession_year_unknown"] = df["accession_year"].isna().astype(int)
print(f"Paintings with unknown accession year: {df['accession_year_unknown'].sum():,} ({df['accession_year_unknown'].mean()*100:.1f}%)")
print(f"Display rate where accession year unknown: {df[df['accession_year_unknown']==1]['is_on_display'].mean():.3f}")

#Defining fill strategies for remaining numeric nulls
#Using department-level medians first for more contextually accurate imputation
#Falling back to global median where department median is unavailable
#Categorical features already handled with 'Unknown' in the functions above

dept_fill_cols = ["height_cm", "width_cm", "area_cm2", "accession_year", "years_in_collection"]

for col in dept_fill_cols:
    if col in df.columns:
        n_missing = df[col].isna().sum()
        if n_missing > 0:
            dept_median = df.groupby("department")[col].transform("median")
            global_median = df[col].median()
            df[col] = df[col].fillna(dept_median).fillna(global_median)
            print(f"  {col}: filled {n_missing} missing using department medians (global fallback: {global_median:,.1f})")

print("\nRemaining nulls in key columns:")
print(df[dept_fill_cols].isnull().sum())

Paintings with unknown accession year: 38 (0.4%)
Display rate where accession year unknown: 0.079
  height_cm: filled 54 missing using department medians (global fallback: 61.3)
  width_cm: filled 85 missing using department medians (global fallback: 53.0)
  area_cm2: filled 85 missing using department medians (global fallback: 3,535.5)
  accession_year: filled 38 missing using department medians (global fallback: 1,979.0)
  years_in_collection: filled 38 missing using department medians (global fallback: 47.0)

Remaining nulls in key columns:
height_cm              0
width_cm               0
area_cm2               0
accession_year         0
years_in_collection    0
dtype: int64


#Seven - Selecting the Final Features for Modelling

Choosing the columns we'll use in exploration and modelling.

In [41]:
#Identifying all tag columns
tag_cols = [c for c in df.columns if c.startswith("tag_")]

# Define our modelling features and metadata columns
id_cols = ["object_id", "title", "artist_name", "object_url"]

# IMPORTANT: gallery_number is intentionally excluded from feature_cols as is_on_display was derived from gallery_number so including it would be data leakage -- the model would simply learn that gallery numbers mean display gallery_number is kept in eda_cols for Tableau use only
# NOTE: is_highlight is included but treat with caution as  Met's editorial highlight flag correlates strongly with display decisions and may partly reflect curatorial judgement made at the same time as display decisions

feature_cols = [
    #Numeric features
    "object_begin_date", "object_end_date", "painting_age", "creation_year",
    "height_cm", "width_cm", "area_cm2",
    "log_area_cm2", "log_height_cm", "log_width_cm",
    "aspect_ratio",
    "accession_year", "years_in_collection",
    "artist_work_count", "tag_count",
    "is_public_domain", "is_highlight",
    # Categorical features
    "department", "has_known_artist", "primary_medium", "support",
    "nationality_group", "acquisition_type", "era", "culture_group",
    # Binary flags
    "is_portrait_orientation", "recently_acquired",
    "is_collaborative_work", "accession_year_unknown",
] + tag_cols

target_col = ["is_on_display"]

#Raw columns are kept for EDA and Tableau only 
# size_band excluded from features as it is derived from area_cm2
# gallery_number excluded from features due to leakage risk (see note above)
eda_cols = ["medium", "culture", "period", "tags", "gallery_number", "credit_line", "size_band"]

all_cols = id_cols + feature_cols + target_col + eda_cols
# Only keep columns that exist
all_cols = [c for c in all_cols if c in df.columns]

df_final = df[all_cols].copy()
print(f"Final dataset: {df_final.shape[0]:,} rows x {df_final.shape[1]} columns")
print(f"\nFeature columns ({len(feature_cols)}): {feature_cols}")

Final dataset: 9,005 rows x 51 columns

Feature columns (40): ['object_begin_date', 'object_end_date', 'painting_age', 'creation_year', 'height_cm', 'width_cm', 'area_cm2', 'log_area_cm2', 'log_height_cm', 'log_width_cm', 'aspect_ratio', 'accession_year', 'years_in_collection', 'artist_work_count', 'tag_count', 'is_public_domain', 'is_highlight', 'department', 'has_known_artist', 'primary_medium', 'support', 'nationality_group', 'acquisition_type', 'era', 'culture_group', 'is_portrait_orientation', 'recently_acquired', 'is_collaborative_work', 'accession_year_unknown', 'tag_men', 'tag_women', 'tag_landscapes', 'tag_portraits', 'tag_mountains', 'tag_trees', 'tag_flowers', 'tag_birds', 'tag_horses', 'tag_abstraction', 'tag_count']


#Eight - Doing a final sense check

In [42]:
df_final.head()

,object_id,title,artist_name,object_begin_date,object_end_date,painting_age,creation_year,height_cm,width_cm,area_cm2,...,tag_abstraction,tag_count,is_on_display,medium,culture,period,tags,gallery_number,credit_line,size_band
0,35155,"Guidobaldo II della Rovere, Duke of Urbino (15...",NaN,1555,1610,444.0,1582.0,14.0,10.2,142.800000,...,0,3,1,Oil on copper,Italian,NaN,Armor|Men|Portraits,374.0,"Purchase, Arthur Ochs Sulzberger Gift, 2009",Small
1,35968,清 佚名 台南地區荷蘭城堡|Forts Zeelandia and Provintia ...,Unidentified artist,1800,1899,176.0,1850.0,110.6,41.5,1981.456953,...,0,5,0,Wall hanging; ink and color on deerskin,China,NaN,Houses|Cities|Boats|Ships|Maps,NaN,"Gift of J. Pierpont Morgan, 1909",NaN
2,35969,NaN,Jin Zunnian,1732,1732,294.0,1732.0,170.2,96.5,16424.300000,...,0,1,0,Hanging scroll; ink and color on silk,China,Qing dynasty (1644–1911),Parrots,NaN,"Rogers Fund, 1912",Very Large
3,35970,明 丁雲鵬 潯陽送客圖 軸|Song of the Lute,Ding Yunpeng,1585,1585,441.0,1585.0,141.3,46.0,6499.800000,...,0,3,0,Hanging scroll; ink and color on paper,China,late Ming dynasty (1368–1644),Mountains|Trees|Boats,NaN,"John Stewart Kennedy Fund, 1913",Large
4,35971,清 佚名 倣王翬 倣李成山水圖 軸|Landscape after Li C...,Wang Hui|Unidentified artist,1700,1911,220.0,1806.0,36.8,28.3,1041.440000,...,0,2,0,Hanging scroll; ink on silk,China,Qing dynasty (1644–1911),Mountains|Landscapes,NaN,"John Stewart Kennedy Fund, 1913",Small


In [43]:
df_final.describe()

,object_id,object_begin_date,object_end_date,painting_age,creation_year,height_cm,width_cm,area_cm2,log_area_cm2,log_height_cm,...,tag_portraits,tag_mountains,tag_trees,tag_flowers,tag_birds,tag_horses,tag_abstraction,tag_count,is_on_display,gallery_number
count,9005.000000,9005.000000,9005.000000,9005.000000,9005.000000,9005.000000,9005.000000,9.005000e+03,8920.000000,8951.000000,...,9005.000000,9005.000000,9005.000000,9005.000000,9005.000000,9005.000000,9005.000000,9005.000000,9005.000000,1379.000000
mean,319351.562132,1734.224653,1775.554692,269.176569,1756.823431,78.063503,86.041997,9.193619e+03,8.102193,4.062249,...,0.105053,0.044753,0.038423,0.037868,0.036979,0.036091,0.034203,1.971349,0.153137,692.311820
std,246756.344754,243.280380,221.412423,221.609772,221.609772,61.222077,120.243635,2.200828e+04,1.482836,0.831261,...,0.306638,0.206772,0.192226,0.190887,0.188722,0.186527,0.181761,1.590754,0.360140,165.870624
min,35155.000000,0.000000,0.000000,4.000000,15.000000,0.260000,0.400000,1.500000e+01,2.772589,0.231112,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,53231.000000,1600.000000,1650.000000,93.000000,1640.000000,30.200000,33.020065,1.045500e+03,6.951033,3.431483,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,612.000000
50%,436539.000000,1803.000000,1858.000000,199.000000,1827.000000,61.300000,52.700000,3.521790e+03,8.170900,4.131961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,638.000000
75%,484154.000000,1921.000000,1936.000000,386.000000,1933.000000,108.900000,92.100000,8.928000e+03,9.113058,4.702297,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,813.000000
max,895946.000000,2022.000000,2059.000000,2011.000000,2022.000000,751.800000,2149.390100,1.136195e+06,13.943197,6.623800,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,1.000000,962.000000


In [44]:
#Confirming no nulls in feature columns
feature_nulls = df_final[feature_cols].isnull().sum()
if feature_nulls.sum() == 0:
    print("No missing values in feature columns.")
else:
    print("Remaining missing values in features:")
    print(feature_nulls[feature_nulls > 0])

Remaining missing values in features:
log_area_cm2     85
log_height_cm    54
log_width_cm     85
aspect_ratio     85
dtype: int64


In [45]:
#Target variable
print(f"Target distribution:")
print(df_final["is_on_display"].value_counts())
print(f"Display rate: {df_final['is_on_display'].mean()*100:.1f}%")

Target distribution:
is_on_display
0    7626
1    1379
Name: count, dtype: int64
Display rate: 15.3%


In [46]:
#Checking for and cleaning html entities or special characters in key text columns
from html import unescape

text_cols_to_clean = ["title", "medium", "culture", "period", "artist_name"]

for col in text_cols_to_clean:
    if col in df.columns:
        html_pattern = df[col].dropna().str.contains(r'&amp;|&lt;|&gt;|&#|<[a-z]', regex=True, na=False)
        count = html_pattern.sum()
        if count > 0:
            print(f"  {col}: found {count} entries with HTML entities -- cleaning now")
            df[col] = df[col].apply(lambda x: unescape(x) if isinstance(x, str) else x)
        else:
            print(f"  {col}: no HTML entities found")

#Verifying that the cleaning was successful
print("\nPost-clean check:")
for col in text_cols_to_clean:
    if col in df.columns:
        remaining = df[col].dropna().str.contains(r'&amp;|&lt;|&gt;|&#', regex=True, na=False).sum()
        print(f"  {col}: {remaining} remaining")

  title: no HTML entities found
  medium: no HTML entities found
  culture: no HTML entities found
  period: no HTML entities found
  artist_name: no HTML entities found

Post-clean check:
  title: 0 remaining
  medium: 0 remaining
  culture: 0 remaining
  period: 0 remaining
  artist_name: 0 remaining


In [47]:
#Now checking the data types and are booleans actually booleans?
print("Data types for key columns:")
for col in ["is_public_domain", "is_highlight", "is_on_display", "accession_year",
            "object_begin_date", "object_end_date", "height_cm", "width_cm"]:
    print(f"  {col}: {df[col].dtype} (sample: {df[col].iloc[0]})")

Data types for key columns:
  is_public_domain: bool (sample: True)
  is_highlight: bool (sample: False)
  is_on_display: int64 (sample: 1)
  accession_year: float64 (sample: 2009.0)
  object_begin_date: int64 (sample: 1555)
  object_end_date: int64 (sample: 1610)
  height_cm: float64 (sample: 14.0)
  width_cm: float64 (sample: 10.2)


In [48]:
#Checking for outliers in numeric columns
print("Numeric column ranges:")
for col in ["painting_age", "creation_year", "area_cm2", "height_cm", "width_cm",
            "years_in_collection", "accession_year", "artist_work_count"]:
    print(f"  {col}: min={df[col].min():.1f}, max={df[col].max():.1f}, median={df[col].median():.1f}")

#Flagging any suspicious values
print(f"\nPaintings with negative age: {(df['painting_age'] < 0).sum()}")
print(f"Paintings dated before year 0: {(df['creation_year'] < 0).sum()}")
print(f"Paintings with area > 100,000 cm2 (very large): {(df['area_cm2'] > 100000).sum()}")

Numeric column ranges:
  painting_age: min=4.0, max=2011.0, median=199.0
  creation_year: min=15.0, max=2022.0, median=1827.0
  area_cm2: min=15.0, max=1136195.3, median=3521.8
  height_cm: min=0.3, max=751.8, median=61.3
  width_cm: min=0.4, max=2149.4, median=52.7
  years_in_collection: min=3.0, max=155.0, median=47.0
  accession_year: min=1871.0, max=2023.0, median=1979.0
  artist_work_count: min=0.0, max=344.0, median=2.0

Paintings with negative age: 0
Paintings dated before year 0: 0
Paintings with area > 100,000 cm2 (very large): 43


#Eight - Outlier Capping

The sense check above confirmed extreme values exist in size and artist count columns. While a random forest handles outliers naturally, capping at the 99th percentile makes summary statistics and Tableau visualisations cleaner and more readable.

In [49]:
#Capping extreme values at the 99th percentile
for col in ["area_cm2", "artist_work_count", "aspect_ratio", "log_area_cm2"]:
    if col in df.columns:
        cap = df[col].quantile(0.99)
        n_capped = (df[col] > cap).sum()
        df[col] = df[col].clip(upper=cap)
        print(f"  {col}: capped {n_capped} values above {cap:,.1f}")

#Rebuilding df_final after capping so the clean file reflects the updated values
df_final = df[all_cols].copy()
print("\ndf_final rebuilt after capping")

  area_cm2: capped 91 values above 76,506.5
  artist_work_count: capped 0 values above 344.0
  aspect_ratio: capped 90 values above 15.4
  log_area_cm2: capped 90 values above 11.2

df_final rebuilt after capping


In [50]:
#Checking the above extreme values - as 21m wide is extreme
print("Paintings with creation_year = 0:")
print(df[df["creation_year"] == 0][["title", "object_begin_date", "object_end_date", "creation_year"]].head(10))
print(f"Total: {(df['creation_year'] == 0).sum()}")

print("\nTop 10 largest paintings by area:")
print(df.nlargest(10, "area_cm2")[["title", "height_cm", "width_cm", "area_cm2", "dimensions"]])

Paintings with creation_year = 0:
Empty DataFrame
Columns: [title, object_begin_date, object_end_date, creation_year]
Index: []
Total: 0

Top 10 largest paintings by area:
                                                  title  height_cm   width_cm  \
964   清  徐揚 等  乾隆南巡圖 （第六卷﹕大運河至蘇州） 卷|The Qianlong Emp...   70.48514  1994.0000   
1015  元　廣勝寺　藥師佛法會圖壁畫|Buddha of Medicine Bhaishajyagu...  751.80000  1511.3000   
1037                    列子図襖|The Daoist Immortal Liezi   182.90000   731.5000   
1039  琴棋書画図襖の内|Appreciation of Painting, from a set ...  182.89990   731.5215   
1157  舞楽図屏風 ・唐獅子図屏風|Bugaku Dances (front); Chinese L...  183.30000   445.4000   
1590  清　王翬等　康熙南巡圖　（卷三: 濟南至泰山）　卷|The Kangxi Emperor's...   67.90000  1393.8000   
1619   清   袁江   九成宮圖   屏|The Palace of Nine Perfections  207.00000   563.2000   
2385                                                NaN  227.30000   375.9000   
2780                                                NaN  184.15040   509.2710   
3120              

In [51]:
Above shows these are legitiimate paintings and murals. So no further action. But want to check the year 0 creation issue 

SyntaxError: invalid syntax (2183653478.py, line 1)

In [ ]:
print(f"Paintings with creation_year = 0: {(df['creation_year'] == 0).sum()}")
print(df[df["creation_year"] == 0][["title", "object_begin_date", "object_end_date"]].head(10))

Paintings with creation_year = 0: 9
                               title  object_begin_date  object_end_date
2307                             NaN                  0                0
2856                             NaN                  0                0
7294                        Headland                  0                0
7472                 Michael Stiener                  0                0
7899         Deer-Sketch from Nature                  0                0
7981  Lydia Crocheting in the Garden                  0                0
7982             The Wyndham Sisters                  0                0
7983          Cremorne Gardens No. 2                  0                0
8459                             NaN                  0                0


In [ ]:
After checking these painintgs they are n19th cent. So Met system puts 0 when it doesn't know year. Adding extra step to painting age cell.

In [ ]:
#And now checking if boolean columns are numeric (noting they are needed for modelling)
for col in ["is_public_domain", "is_highlight"]:
    if df[col].dtype == "bool" or df[col].dtype == "object":
        df[col] = df[col].astype(int)
        print(f"  Converted {col} to int")
    else:
        print(f"  {col} already numeric ({df[col].dtype})")

  Converted is_public_domain to int
  Converted is_highlight to int


In [ ]:
#Final check for any remaining nulls in feature columns
print("Remaining nulls in all columns:")
nulls = df.isnull().sum()
nulls = nulls[nulls > 0]
if len(nulls) == 0:
    print("  None — all clean")
else:
    print(nulls.sort_values(ascending=False))

Remaining nulls in all columns:
gallery_number        7626
period                6274
culture               4581
title                 2230
tags                  1764
artist_name            984
artist_nationality     984
artist_begin_date      984
artist_end_date        984
dimensions              14
credit_line              5
medium                   4
dtype: int64


#Summary of the additional data quality checks following part eight

Before finalising the clean dataset, the following checks were carried out:

1. HTML entities - checked across title, medium, culture, period and artist_name columns. Any html encoding found was cleaned using Python's built-in unescape function.
2. Data types - converted is_public_domain and is_highlight from boolean to integer for model compatibility.
3. Outliers - 43 paintings with area over 100,000 cm2 were inspected and confirmed as legitimate large-scale works such as murals and installations. Extreme values were then capped at the 99th percentile across area, artist work count, aspect ratio and log area to keep summary statistics and visualisations clean.
4. Invalid dates - paintings with creation_year of 0 were filled with the department-level median creation year rather than the global median, for greater accuracy across the collection's diverse time periods.
5. Data leakage check - gallery_number confirmed as excluded from feature_cols. It is kept in eda_cols for Tableau use only. is_highlight flagged as a feature to treat with caution in model interpretation.
6. Remaining nulls - nulls remain only in metadata columns such as title, culture and period which are not used as model features. All feature columns are fully populated.

#Nine - Saving the Clean Dataset

In [ ]:
output_path = "data/clean/met_paintings_clean.csv"
df_final.to_csv(output_path, index=False)
print(f"Saved {len(df_final):,} paintings to {output_path}")
print(f"File size: {os.path.getsize(output_path) / (1024*1024):.1f} MB")

Saved 9,005 paintings to data/clean/met_paintings_clean.csv
File size: 2.7 MB


#Summary of Stage Two Work

In this notebook I have done the following:
1. Removed duplicate rows from the merge step
2. Parsed dimension strings into numeric height/width where API data was missing, with an inches-to-cm conversion fallback for pure-inches strings
3. Extracted primary medium (e.g. Oil, Ink, Watercolour, etc.) and support (Canvas, Silk, Paper, etc.)
4. Grouped artist nationalities and culture values into manageable regional categories
5. Engineered new features: painting age, years in collection, area, log-transformed size features, aspect ratio, orientation, artist work count, tag dummies
6. Added era buckets grouping each painting into a named historical period based on creation year
7. Added a recently acquired flag for paintings entering the collection from 1990 onwards
8. Added an accession_year_unknown flag to preserve the signal that very early acquisitions carry no year record
9. Added a collaborative work flag for paintings attributed to multiple artists
10. Added analytical summaries of median size and creation year by department and medium, and display rate by size band
11. Extracted acquisition type from credit line
12. Collapsed rare categories (under 50 paintings) into Other across all categorical columns
13. Handled remaining missing values with department-level median imputation, falling back to global median where needed
14. Capped outliers at the 99th percentile in skewed numeric columns
15. Confirmed gallery_number is excluded from feature_cols to prevent data leakage
16. Saved a clean dataset ready for EDA and modelling

#Next Step: `03_eda_and_sql.ipynb` - exploratory data analysis using SQL and pandas.

In [53]:
import os

path = "data/clean/met_paintings_clean.csv"
if os.path.exists(path):
    df_check = pd.read_csv(path)
    print(f"Rows: {df_check.shape[0]:,}")
    print(f"Columns: {df_check.shape[1]}")
    print(f"Display rate: {df_check['is_on_display'].mean()*100:.1f}%")
    print(f"has_known_artist in columns: {'has_known_artist' in df_check.columns}")
else:
    print("File not found - so run the save cell again in Part Nine")

Rows: 9,005
Columns: 40
Display rate: 15.3%
has_known_artist in columns: True


In [58]:
# Create missing columns
df['area_quartile'] = pd.qcut(df['area_cm2'], q=4, labels=['Q1 (smallest)', 'Q2', 'Q3', 'Q4 (largest)'])

df['collection_age_band'] = pd.cut(df['creation_year'], 
    bins=[0, 1400, 1600, 1800, 1900, 1950, 2000, 2100],
    labels=['Pre-1400', '1400-1600', '1600-1800', '1800-1900', '1900-1950', '1950-2000', 'Post-2000'])

# Export
tableau_export = df[[
    'object_id',
    'department',
    'acquisition_type',
    'area_cm2',
    'area_quartile',
    'collection_age_band',
    'is_on_display',
    'creation_year'
]].copy()

tableau_export.to_csv('data/clean/met_tableau_export.csv', index=False)
print(f"Exported {len(tableau_export)} rows")
print(tableau_export.head())

Exported 9005 rows
   object_id      department acquisition_type      area_cm2  area_quartile  \
0      35155  Arms and Armor             Gift    142.800000  Q1 (smallest)   
1      35968       Asian Art             Gift   1981.456953             Q2   
2      35969       Asian Art    Purchase/Fund  16424.300000   Q4 (largest)   
3      35970       Asian Art    Purchase/Fund   6499.800000             Q3   
4      35971       Asian Art    Purchase/Fund   1041.440000  Q1 (smallest)   

  collection_age_band  is_on_display  creation_year  
0           1400-1600              1         1582.0  
1           1800-1900              0         1850.0  
2           1600-1800              0         1732.0  
3           1400-1600              0         1585.0  
4           1800-1900              0         1806.0  
